In [11]:
import pandas as pd
import numpy as np
import sqlite3

In [12]:
def load_day(day):
    header = ['timestamp', 'line_id', 'direction', 'jrny_patt_id', 'time_frame', 'journey_id', 'operator', 
              'congestion', 'lon', 'lat', 'delay', 'block_id', 'vehicle_id', 'stop_id', 'at_stop']
    types = {'timestamp': np.int64,
             'journey_id': np.int32,
             'congestion': np.int8,
             'lon': np.float64,
             'lat': np.float64,
             'delay': np.int8,
             'vehicle_id': np.int32,
             'at_stop': np.int8}
    file_name = 'data/siri.201301{0:02d}.csv'.format(day)
    df = pd.read_csv(file_name, header=None, names=header, dtype=types, parse_dates=['time_frame'], infer_datetime_format=True)
    null_replacements = {'line_id': 0, 'stop_id': 0}
    df = df.fillna(value=null_replacements)
    df['line_id'] = df['line_id'].astype(np.int32)
    df['stop_id'] = df['stop_id'].astype(np.int32)
    # df['timestamp'] = pd.to_datetime(df['timestamp'], unit='us')
    return df

In [13]:
def haversine_np(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    
    Taken from here: https://stackoverflow.com/questions/29545704/fast-haversine-approximation-python-pandas#29546836
    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    #c = 2 * np.arcsin(np.sqrt(a))
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1.0 - a))
    meters = 6372000.0 * c
    return meters

In [14]:
def calculate_durations(data_frame, vehicle_id):
    one_second = np.timedelta64(1000000000, 'ns')
    dv = data_frame[data_frame['vehicle_id']==vehicle_id]
    ts = dv.timestamp.values
    dtd = ts[1:] - ts[:-1]
    dt = np.zeros(len(dtd) + 1)
    dt[1:] = dtd / one_second
    return dt

In [15]:
def calculate_distances(data_frame, vehicle_id):
    dv = data_frame[data_frame['vehicle_id']==vehicle_id]
    lat = dv.lat.values
    lon = dv.lon.values
    dxm = haversine_np(lon[1:], lat[1:], lon[:-1], lat[:-1])
    dx = np.zeros(len(dxm) + 1)
    dx[1:] = dxm
    return dx

In [16]:
def filter_columns(df):
    columns = ['timestamp', 'direction', 'journey_id', 'congestion', 'lon', 'lat', 'delay', 'vehicle_id', 'stop_id', 'at_stop']
    return df[columns]

In [17]:
def create_table(conn):
    c = conn.cursor()
    c.execute("CREATE TABLE locations (id integer primary key, timestamp, direction, journey_id, congestion, lon, lat, vehicle_id, stop_id, at_stop)")
    conn.commit()

In [18]:
def insert_row(conn, row):
    c = conn.cursor()
    parameters = (row['timestamp'], row['direction'], row['journey_id'], row['congestion'], row['lon'], row['lat'], row['vehicle_id'], row['stop_id'], row['at_stop'])
    c.execute("insert into locations (timestamp, direction, journey_id, congestion, lon, lat, vehicle_id, stop_id, at_stop) values (?,?,?,?,?,?,?,?,?)", parameters)
    conn.commit()

In [19]:
conn = sqlite3.connect("data/dublin-bus.db")

create_table(conn)

conn.close()

OperationalError: table locations already exists

In [20]:
conn = sqlite3.connect("data/dublin-bus.db")

for d in range(31):
    print("Day {0}".format(d+1))
    
    day = filter_columns(load_day(d+1))
    day['dt'] = 0.0
    day['dx'] = 0.0
    day['speed'] = 0.0
    
    vehicles = day['vehicle_id'].unique() 
    for v in vehicles:
        vehicle_selector = day['vehicle_id']==v
        day.loc[vehicle_selector,'dt'] = calculate_durations(day, v)
        day.loc[vehicle_selector,'dx'] = calculate_distances(day, v)
    
    speed_selector = day['dt'] > 0
    day.loc[speed_selector,'speed'] = day[speed_selector].dx / day[speed_selector].dt * 3.6

    # Filter invalid points (speeds over 100 km/h)
    day = day[day['speed'] < 100.0]
    day.apply(lambda row: insert_row(conn, row))

conn.close()

Day 1


TypeError: ufunc true_divide cannot use operands with types dtype('int64') and dtype('<m8[ns]')